# Случайные процессы. Прикладной поток.
## Семинар 4

In [1]:
import numpy as np
import nltk  # sudo pip3 install nltk
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score

Скачайте `conll2000` следующей командой. Если у вас появляется новое окно, выберите там вкладку `Corpora`

In [ ]:
nltk.download()

In [3]:
from nltk.corpus import conll2000

Разобъем данные на две части --- по первой части оценим параметры, а по второй посчитаем качество.

In [4]:
train_sents = conll2000.tagged_sents()[:8000]
test_sents = conll2000.tagged_sents()[8000:]

Данные представлены в виде списка предложений, где для каждого слова указан его тег (часть речи).

In [5]:
train_sents

[[('Confidence', 'NN'), ('in', 'IN'), ('the', 'DT'), ('pound', 'NN'), ('is', 'VBZ'), ('widely', 'RB'), ('expected', 'VBN'), ('to', 'TO'), ('take', 'VB'), ('another', 'DT'), ('sharp', 'JJ'), ('dive', 'NN'), ('if', 'IN'), ('trade', 'NN'), ('figures', 'NNS'), ('for', 'IN'), ('September', 'NNP'), (',', ','), ('due', 'JJ'), ('for', 'IN'), ('release', 'NN'), ('tomorrow', 'NN'), (',', ','), ('fail', 'VB'), ('to', 'TO'), ('show', 'VB'), ('a', 'DT'), ('substantial', 'JJ'), ('improvement', 'NN'), ('from', 'IN'), ('July', 'NNP'), ('and', 'CC'), ('August', 'NNP'), ("'s", 'POS'), ('near-record', 'JJ'), ('deficits', 'NNS'), ('.', '.')], [('Chancellor', 'NNP'), ('of', 'IN'), ('the', 'DT'), ('Exchequer', 'NNP'), ('Nigel', 'NNP'), ('Lawson', 'NNP'), ("'s", 'POS'), ('restated', 'VBN'), ('commitment', 'NN'), ('to', 'TO'), ('a', 'DT'), ('firm', 'NN'), ('monetary', 'JJ'), ('policy', 'NN'), ('has', 'VBZ'), ('helped', 'VBN'), ('to', 'TO'), ('prevent', 'VB'), ('a', 'DT'), ('freefall', 'NN'), ('in', 'IN'), ('s

Предложение представлено в виде списка пар слово-тег. Пример:

In [6]:
train_sents[0]

[('Confidence', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('pound', 'NN'),
 ('is', 'VBZ'),
 ('widely', 'RB'),
 ('expected', 'VBN'),
 ('to', 'TO'),
 ('take', 'VB'),
 ('another', 'DT'),
 ('sharp', 'JJ'),
 ('dive', 'NN'),
 ('if', 'IN'),
 ('trade', 'NN'),
 ('figures', 'NNS'),
 ('for', 'IN'),
 ('September', 'NNP'),
 (',', ','),
 ('due', 'JJ'),
 ('for', 'IN'),
 ('release', 'NN'),
 ('tomorrow', 'NN'),
 (',', ','),
 ('fail', 'VB'),
 ('to', 'TO'),
 ('show', 'VB'),
 ('a', 'DT'),
 ('substantial', 'JJ'),
 ('improvement', 'NN'),
 ('from', 'IN'),
 ('July', 'NNP'),
 ('and', 'CC'),
 ('August', 'NNP'),
 ("'s", 'POS'),
 ('near-record', 'JJ'),
 ('deficits', 'NNS'),
 ('.', '.')]

Рассмотрим самую простую модель
$$tag(w) = \arg \max_{i \in 1 .. |Tags| } P(tag_i \mid w).$$

В данной модели каждому слову сопоставляется тег, который дает максимум условной вероятности для данного слова. Можно заметить, что такое правило сопоставления тегов не зависит от контекста.
Модель реализуется следующим классом.

*При использовании такой модели не получится сопоставить слову тег, если этого слова не было в размеченных данных. Для повышения качества можно в таких случаях всегда выдавать, например, тег `NN`.*

In [7]:
class ObviousPOSTagger:
    def fit(self, train_sents):
        '''Оценка условных вероятностей по предожениям из train_sents.'''
        
        self.model = defaultdict(Counter)

        # Создание словаря {(слово, тег): количество}
        for t_sent in train_sents:
            for word, tag in t_sent:
                self.model[word][tag] += 1

        # Нормировка (необязательна, т.к. берем argmax)
        #for word in self.model:
        #    sum_values = sum(self.model[word].values())
        #    for token in self.model[word]:
        #        self.model[word][token] /= sum_values
                
                
    def tagging(self, sent):
        '''Каждому слову из предложений sent сопоставляет тег.'''
        
        tags = []
        for word in sent:
            tags.append(self.model[word].most_common(1)[0][0] if word in self.model else 'NN')
        return list(zip(sent, tags))

Определим модель и оценим ее параметры по размеченным данным.

In [8]:
tagger = ObviousPOSTagger()
tagger.fit(train_sents)

С помощью полученной модели определим теги для второй части размеченных данных

In [9]:
tagging_test = [tagger.tagging([word for word, tag in sent]) for sent in test_sents]

Составим список полученных тегов, которые определила модель и которые уже были в наших данных.

In [10]:
correct_tags = [tag for sent in test_sents for word, tag in sent]
predict_tags = [tag for sent in tagging_test for word, tag in sent]

Точность (доля верно предсказанных тегов)

In [13]:
print('%.3f' % accuracy_score(correct_tags, predict_tags))

0.907
